In [ ]:
import os
import nltk
nltk.download('punkt')
import numpy as np
import json
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
from nltk.stem.lancaster import LancasterStemmer
from string import punctuation
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from transformers import DistilBertTokenizer, DistilBertModel
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, LinearRegression
from sklearn.linear_model import Ridge

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
anger_training_set = []
fear_training_set = []
sadness_training_set = []
joy_training_set = []

anger_test_set = []
fear_test_set = []
sadness_test_set = []
joy_test_set = []
stemmer = LancasterStemmer()
all_words=[]

In [ ]:
def load_training_data(sentiment):
    data = open("/content/"+sentiment+"_training_set.txt",encoding="utf8")
    if sentiment == "anger":
        threshold = 0.5
    elif sentiment == "fear":
        threshold = 0.6
    elif sentiment == "sadness":
        threshold = 0.5
    elif sentiment == "joy":
        threshold = 0.5
    else:
        pass
    return data,threshold


In [ ]:
def load_test_data(sentiment):
    data = open("/content/"+sentiment+"_test_set.txt",encoding="utf8")
    return data

In [ ]:
def clean_data(training_data,threshold):
    training_set = []
    for line in training_data:
        line = line.strip().lower()
        if line.split()[-1] == "none":
            line = " ".join(filter(lambda x:x[0]!='@', line.split()))
            punct = line.maketrans("","",'.*%$^0123456789#!][\?&/)/(+-<>')
            result = line.translate(punct)
            tokened_sentence = nltk.word_tokenize(result)
            sentence = tokened_sentence[0:len(tokened_sentence)-1]
            label = tokened_sentence[-2]
            training_set.append((sentence,label))
        else:
            intensity = float(line.split()[-1])
            if (intensity>=threshold):
                line = " ".join(filter(lambda x:x[0]!='@', line.split()))
                punct = line.maketrans("","",'.*%$^0123456789#!][\?&/)/(+-<>')
                result = line.translate(punct)
                tokened_sentence = nltk.word_tokenize(result)
                sentence = tokened_sentence[0:len(tokened_sentence)-1]
                label = tokened_sentence[-1]
                training_set.append((sentence,label))
    return training_set


In [ ]:
def bag_of_words(all_data):
    training_set = []
    all_words = []
    for each_list in all_data:
        for words in each_list[0]:
            word = stemmer.stem(words)
            all_words.append(word)
    all_words = list(set(all_words))

    for each_sentence in all_data:
        bag = [0]*len(all_words)
        training_set.append(encode_sentence(all_words,each_sentence[0],bag))
    return training_set,all_words


In [ ]:
def encode_sentence(all_words,sentence, bag):
    for s in sentence:
        stemmed_word = stemmer.stem(s)
        for i,word in enumerate(all_words):
            if stemmed_word == word:
                bag[i] = 1
    return bag

In [ ]:
def main():
    bag = []
    all_data = []
    all_test_data = []
    labels = []
    classes = []
    labels = []
    test_labels = []
    words=[]
    test_words = []

    
    anger_training_data,threshold = load_training_data("anger")
    anger_training_set = clean_data(anger_training_data,threshold)
    print(anger_training_set[0])

    fear_training_data,threshold = load_training_data("fear")
    fear_training_set = clean_data(fear_training_data,threshold)

    sadness_training_data,threshold = load_training_data("sadness")
    sadness_training_set = clean_data(sadness_training_data,threshold)

    joy_training_data,threshold = load_training_data("joy")
    joy_training_set = clean_data(joy_training_data,threshold)


    anger_test_data = load_test_data("anger")
    anger_test_set = clean_data(anger_test_data,threshold)
    print(anger_test_set[0])
    print(len(anger_test_set))

    fear_test_data = load_test_data("fear")
    fear_test_set = clean_data(fear_test_data,threshold)
    print(fear_test_set[0])
    print(len(fear_test_set))

    sadness_test_data = load_test_data("sadness")
    sadness_test_set = clean_data(sadness_test_data,threshold)
    print(sadness_test_set[0])
    print(len(sadness_test_set))

    joy_test_data = load_test_data("joy")
    joy_test_set = clean_data(joy_test_data,threshold)
    print(joy_test_set[0])
    print(len(joy_test_set))


    all_data.extend(anger_training_set)
    all_data.extend(fear_training_set)
    all_data.extend(sadness_training_set)
    all_data.extend(joy_training_set)

    all_data.extend(anger_test_set)
    all_data.extend(fear_test_set)
    all_data.extend(sadness_test_set)
    all_data.extend(joy_test_set)



    for i,j in all_data:
        if j == "anger":
            labels.append([1,0,0,0])
        elif j == "fear":
            labels.append([0,1,0,0])
        elif j == "sadness":
            labels.append([0,0,1,0])
        elif j == "joy":
            labels.append([0,0,0,1])
        else:
            pass

    print(len(labels))
    print(len(test_labels))
    classes = ["anger","fear","sadness","joy"]
    print(classes)
    np.set_printoptions(threshold=np.inf)


    training_set,words = bag_of_words(all_data)

    dataset = np.array(training_set)
    labels = np.array(labels)


    shuffling_function = np.random.permutation(dataset.shape[0])
    shuffled_dataset, shuffled_labels = np.zeros((dataset.shape)),np.zeros((dataset.shape))
    shuffled_dataset,shuffled_labels = dataset[shuffling_function],labels[shuffling_function]


    split = int(len(shuffled_dataset)*0.8)
    training_data = shuffled_dataset[:split]
    training_labels = shuffled_labels[:split]
    test_data = shuffled_dataset[split:]
    test_labels = shuffled_labels[split:]
    print(training_data.shape)
    print(training_labels.shape)
    print(test_data.shape)
    print(test_labels.shape)


    Train_model(training_data,training_labels,words,classes)
    Test_model(test_data,test_labels,words,classes)


In [ ]:
def sigmoid(z):
    return (1/(1+np.exp(-z)))

def relu(z):
    A = np.array(z,copy=True)
    A[z<0]=0
    assert A.shape == z.shape
    return A

def softmax(x):
    num = np.exp(x-np.amax(x,axis=0,keepdims=True))
    return num/np.sum(num,axis=0,keepdims=True)


In [ ]:

def forward_prop(n_x,n_h,n_y,m,X,W1,W2,b1,b2):
    Z1 = np.dot(W1,X)+b1
    A1 = relu(Z1)
    Z2 = np.dot(W2,A1)+b2
    A2 = softmax(Z2)
    return Z1,A1,Z2,A2

def relu_backward(da,dz):
    da1 = np.array(da,copy=True)
    da1[dz<0]=0
    assert da1.shape == dz.shape
    return da1

def linear_backward(dz,a,m,w,b):
    dw = (1/m)*np.dot(dz,a.T)
    db = (1/m)*np.sum(dz,axis=1,keepdims=True)
    da = np.dot(w.T,dz)
    assert (dw.shape==w.shape)
    assert (da.shape==a.shape)
    assert (db.shape == b.shape)
    return da,dw,db


def calculate_loss(Y,Yhat,m):
    loss = (-1/m)*np.sum(np.multiply(Y,np.log(Yhat)))
    return loss


def back_prop(Z1,A1,Z2,A2,X,Y,W1,W2,b1,b2,learning_rate,m):
    dZ2 = A2-Y
    da1,dw2,db2 = linear_backward(dZ2,A1,m,W2,b2)
    dZ1 = relu_backward(da1,Z1)
    da0,dw1,db1 = linear_backward(dZ1,X,m,W1,b1)
    W2 = W2 - learning_rate * dw2
    b2 = b2 - learning_rate * db2
    W1 = W1 - learning_rate * dw1
    b1 = b1 - learning_rate * db1
    return W1,b1,W2,b2



In [ ]:


def Test_model(test_data, test_labels,words,classes):
    all_losses = []
    learning_rate = 0.1
    iterations = 50
    np.random.seed(1)
    X = test_data.T
    print(" Shape of X is ", X.shape)
    Y = test_labels.T
    print(" Shape of Y is ", Y.shape)

    m = X.shape[1]
    print(" Shape of m is ", m)
   
    n_h = 100
  
    n_x = X.shape[0]

    n_y = 4

    weights_file = 'weights.json'
    with open(weights_file) as data_file:
        weights = json.load(data_file)
        W1 = np.asarray(weights['weight1'])
        W2 = np.asarray(weights['weight2'])
        b1 = np.asarray(weights['bias1'])
        b2 = np.asarray(weights['bias2'])

    print("################### TEST MODEL STATISTICS ######################")
    for i in range(1):
        
        l0 = X
       
        l1 = relu(np.dot(W1,l0)+b1)
        # output layer
        l2 = softmax(np.dot(W2,l1)+b2)
        predictions = np.argmax(l2, axis=0)
        labels = np.argmax(Y, axis=0)
        print(classification_report(predictions,labels))




In [ ]:


def Train_model(training_data, training_labels,words,classes):
    all_losses = []
    learning_rate = 0.1
    iterations = 50
    np.random.seed(1)
    X = training_data.T
    print(" Shape of X is ", X.shape)
    Y = training_labels.T
    print(" Shape of Y is ", Y.shape)
    
    m = X.shape[1]
    print(" Shape of m is ", m)
    
    n_h = 100
    
    n_x = X.shape[0]
   
    n_y = 4
    
    W1 = np.random.randn(n_h,n_x)*0.01
    print(" Shape of W1 is ", W1.shape)
   
    b1 = np.zeros((n_h,1))
    
    W2 = np.random.randn(n_y,n_h)
    print(" Shape of W2 is ", W2.shape)
   
    b2 = np.zeros((n_y,1))
    print("################### TRAIN MODEL STATISTICS ######################")
    for i in range(0,iterations):
        Z1,A1,Z2,A2 = forward_prop(n_x,n_h,n_y,m,X,W1,W2,b1,b2)
        predictions = np.argmax(A2, axis=0)
        labels = np.argmax(Y, axis=0)
        print(classification_report(predictions,labels))
        Loss = calculate_loss(Y,A2,m)
        W1,b1,W2,b2 = back_prop(Z1,A1,Z2,A2,X,Y,W1,W2,b1,b2,learning_rate,m)
        all_losses.append(Loss)

    
    weights = {'weight1': W1.tolist(), 'weight2': W2.tolist(),
               'bias1':b1.tolist(), 'bias2':b2.tolist(),
               'words': words,
               'classes': classes
              }
    weights_file = "weights.json"

    with open(weights_file, 'w') as outfile:
        json.dump(weights, outfile, indent=4, sort_keys=True)
    print ("saved synapses to:", weights_file)
    plt.plot(all_losses)



In [ ]:
if __name__ == '__main__':
    main()

In [ ]:

ERROR_THRESHOLD = 0.1

weights_file = 'weights.json'
with open(weights_file) as data_file:
    weights = json.load(data_file)
    W1 = np.asarray(weights['weight1'])
    W2 = np.asarray(weights['weight2'])
    b1 = np.asarray(weights['bias1'])
    b2 = np.asarray(weights['bias2'])
    all_words = weights['words']
    classes = weights['classes']

def clean_sentence(verification_data):
    line = verification_data
   
    line = line.strip().lower()
    
    line = " ".join(filter(lambda x:x[0]!='@', line.split()))
   
    punct = line.maketrans("","",'.*%$^0123456789#!][\?&/)/(+-<>')
    result = line.translate(punct)
    
    tokened_sentence = nltk.word_tokenize(result)
    
    sentence = tokened_sentence[0:len(tokened_sentence)]
    return sentence

def verify(sentence, show_details=False):
    bag=[0]*len(all_words)
    cleaned_sentence = clean_sentence(sentence)
   
    x = encode_sentence(all_words,cleaned_sentence,bag)
    x = np.array(x)
    x = x.reshape(x.shape[0],1)

    print("Shape of X is ", x.shape)
    if show_details:
        print ("sentence:", sentence, "\n bow:", x)
    
    l0 = x
   
    l1 = relu(np.dot(W1,l0)+b1)
    
    l2 = softmax(np.dot(W2,l1)+b2)

    return l2

def classify(sentence, show_details=False):
    results = verify(sentence, show_details)
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD ]
    results.sort(key=lambda x: x[1], reverse=True)
    return_results =[[classes[r[0]],r[1]] for r in results]
    print ("%s \n classification: %s \n" % (sentence, return_results))
    return return_results

classify("I want to kill everyone @Name1 #why?")
classify("I am so happy @Name2 #yayyyy")
classify("This depression will kill me someday .. i am dying @Name3 #killme")
classify("I am afraid terrorists might attack us @Name4 #isis")
classify("What should I do when i am happy @Name5 ")
classify("I want to be happy")

In [ ]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Attention
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, DecisionTreeRegressor, StackingRegressor
from sklearn.linear_model import Ridge
from sklearn.svm import SVR


df = pd.read_csv("data.csv")


tokenizer = Tokenizer()
tokenizer.fit_on_texts(df["text"])


sequences = tokenizer.texts_to_sequences(df["text"])


max_seq_len = max(len(s) for s in sequences)
sequences = pad_sequences(sequences, maxlen=max_seq_len)


X_train, X_test, y_train, y_test = train_test_split(sequences, df["emotion"], test_size=0.2)


model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index), output_dim=128))
model.add(LSTM(128))
model.add(Attention(attention_mechanism='bahdanau'))
model.add(Dense(len(df["emotion"].unique())))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


model.fit(X_train, y_train, epochs=10)


y_pred = model.predict(X_test)


intensity_labels = {
    0: "Anger",
    1: "Fear",
    2: "sad",
    3: "Joy"
}


model = StackingRegressor([
    ('rf', RandomForestRegressor()),
    ('svr', SVR()),
    ('ridge', Ridge()),
    ('dt', DecisionTreeRegressor())
])


model.fit([y_pred, y_test], y_test)


y_pred_intensity = model.predict([y_pred, y_test])


print("Emotion classification accuracy:", model.evaluate(X_test, y_test)[1])
print("Emotion intensity prediction RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_intensity)))


In [ ]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Attention
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, DecisionTreeRegressor, StackingRegressor
from sklearn.linear_model import Ridge
from sklearn.svm import SVR


df = pd.read_csv("data.csv")


tokenizer = Tokenizer()
tokenizer.fit_on_texts(df["text"])


sequences = tokenizer.texts_to_sequences(df["text"])


max_seq_len = max(len(s) for s in sequences)
sequences = pad_sequences(sequences, maxlen=max_seq_len)


X_train, X_test, y_train, y_test = train_test_split(sequences, df["emotion"], test_size=0.2)


model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index), output_dim=128,
                   embeddings_initializer='distilbert', trainable=False))
model.add(LSTM(128))
model.add(Attention(attention_mechanism='bahdanau'))
model.add(Dense(len(df["emotion"].unique())))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


model.fit(X_train, y_train, epochs=10)


y_pred = model.predict(X_test)


intensity_labels = {
    0: "anger",
    1: "sad",
    2: "fear",
    3: "joy"
}


model = StackingRegressor([
    ('rf', RandomForestRegressor()),
    ('svr', SVR()),
    ('ridge', Ridge()),
    ('dt', DecisionTreeRegressor())
])


combined_features = np.hstack((y_pred, y_test))


model.fit(combined_features, y_test)


y_pred_intensity = model.predict(combined_features)


print("Emotion classification accuracy:", model.evaluate(X_test, y_test)[1])
print("Emotion intensity prediction RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_intensity)))


In [ ]:

emotion_data = pd.read_csv('emotion_data.csv')  
X_text = emotion_data['text']
y_emotion = emotion_data['emotion']


X_train, X_test, y_train_emotion, y_test_emotion = train_test_split(X_text, y_emotion, test_size=0.2, random_state=42)


tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
X_train_tokens = tokenizer.batch_encode_plus(X_train.tolist(), padding=True, truncation=True, return_tensors='pt')
X_test_tokens = tokenizer.batch_encode_plus(X_test.tolist(), padding=True, truncation=True, return_tensors='pt')


model = DistilBertModel.from_pretrained('distilbert-base-uncased', output_hidden_states=True)
X_train_embeddings = model(**X_train_tokens)['hidden_states'][-2].mean(dim=1)
X_test_embeddings = model(**X_test_tokens)['hidden_states'][-2].mean(dim=1)


emotion_classifier = model()
emotion_classifier.fit(X_train_embeddings.detach().numpy(), y_train_emotion)


y_pred_emotion = emotion_classifier.predict(X_test_embeddings.detach().numpy())




intensity_data = pd.read_csv('intensity_data.csv')
X_intensity = intensity_data['text']
y_intensity = intensity_data['intensity']


X_train_intensity, X_test_intensity, y_train_intensity, y_test_intensity = train_test_split(
    X_intensity, y_intensity, test_size=0.2, random_state=42
)


X_train_combined = np.column_stack((X_train_embeddings.detach().numpy(), y_pred_emotion))
X_test_combined = np.column_stack((X_test_embeddings.detach().numpy(), y_pred_emotion))


ensemble_models = [
    DecisionTreeRegressor(),
    SVR(),
    RandomForestRegressor(),
    Ridge()
]


ensemble_predictions_train = []
ensemble_predictions_test = []
for model in ensemble_models:
    model.fit(X_train_combined, y_train_intensity)
    ensemble_predictions_train.append(model.predict(X_train_combined))
    ensemble_predictions_test.append(model.predict(X_test_combined))

X_train_stacked = np.column_stack(ensemble_predictions_train)
X_test_stacked = np.column_stack(ensemble_predictions_test)


final_model = LinearRegression()
final_model.fit(X_train_stacked, y_train_intensity)


y_pred_intensity = final_model.predict(X_test_stacked)


rmse = np.sqrt(mean_squared_error(y_test_intensity, y_pred_intensity))
print("RMSE:", rmse)


In [ ]:
from IPython.display import clear_output
input_sentiment = input("Hi :) How are you feeling today ? ")
print(input_sentiment)
print(classify(input_sentiment)[0][0])
sentiment = classify(input_sentiment)[0][0]
print(sentiment)
if sentiment == "anger" or sentiment == "sadness" or sentiment == "fear":
    answer = input("Sorry to hear that .... would you like to hear a joke to lighten your mood ? Press Yes or No ")
    if answer == "N" or answer == "No" or answer == "no" or answer == "n":
        print("Have a nice day. Goodbye :) ")
    else:
        file = open('content/jokes.txt','r')
        while(1):
            full_file = file.readline()
            split_file = full_file.split('/')
            print(split_file)
            slashes = full_file.count('/')
            print(slashes)
            line_of_joke = []
            for i in range(slashes):
                k=0
                print(split_file[i])
                commas = split_file[i].count('"')
                print(commas)
                length = int(commas/2)
                if length == 0:
                    line_of_joke.append(split_file[i])
                else:
                    for j in range(length):
        
                        line_of_joke.append(split_file[i].split('"')[k]+split_file[i].split('"')[k+1])
                        if j==length-1:
                            line_of_joke.append(split_file[i].split('"')[k+2])
                        k=k+2
  
            for i in line_of_joke:
                print(i)
            user_input = input("Do you want another joke ? Write Yes or No\t")
            if user_input == "Y":
                clear_output()
                pass
            else:
                clear_output()
                break
        print(line_of_joke[1])

In [ ]:
from IPython.display import clear_output
input_sentiment = input("today is a good day")
print(input_sentiment)
print(classify(input_sentiment)[0][0])
sentiment = classify(input_sentiment)[0][0]
print(sentiment)
line=[]
        file = open('content/model.txt','r')
        while(1):
            full_file = file.readline()
            split_file = full_file.split('/')
            print(split_file)
            slashes = full_file.count('/')
            print(slashes)

            for i in range(slashes):
                k=0
               print(split_file[i])
                commas = split_file[i].count('"')
               print(commas)
                length = int(commas/2)
                if length == 0:
                    line.append(split_file[i])
                else:
                    for j in range(length):
                    print("Here")
                        line.append(split_file[i].split('"')[k]+split_file[i].split('"')[k+1])
                        if j==length-1:
                            line.append(split_file[i].split('"')[k+2])
                        k=k+2